In [1]:
import pandas as pd
from cleantext import clean
import re
import torch.nn.functional as F


import torch
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
from pandas import read_excel, read_json
import sys
import time
import math

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
/home/remi/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
window_size = 310
slide = 70


In [3]:
def make_windows(content, w_size, slide):
    if slide > w_size :
        raise Exception()
    tokens = content.split(" ")
    l = len(tokens)    
       
    n = math.ceil(l / (w_size-slide))        

    if n==1 :
        return [content]
    else :
        windows = []
        for i in range(n) :            
            x = i*slide
            if x < l :
                y = x + w_size
                w = tokens[x:y]
                windows.append(" ".join(w))
        
        return windows

In [4]:
contribs = pd.read_csv("/home/remi/Downloads/xp-documents-contribs (1).csv")

In [5]:
contribs.drop_duplicates(subset=["answers.generic.text"], inplace=True)
contribs["cleaned_text"] = contribs["answers.generic.text"].apply(lambda t : clean(re.sub("\s+", " ", str(t)), to_ascii=False))
contribs["tokens"] = contribs["cleaned_text"].apply(lambda t : len(str(t).split(" ")))
contribs["windows"] = contribs["cleaned_text"].apply(lambda t : make_windows(t, window_size, slide))
contribs.drop(columns=["cleaned_text","answers.generic.text","text"], inplace=True)
contribs = contribs.explode("windows", ignore_index=True)
contribs.rename(columns={"windows": "text"}, inplace=True)

In [6]:
contribs

,source,cdtnId,title,slug,tokens,text
0,contributions,10056a88d1,Quelle est la durée de préavis en cas de mise ...,2542-quelle-est-la-duree-de-preavis-en-cas-de-...,250,"en cas de mise à la retraite, le salarié effec..."
1,contributions,10056a88d1,Quelle est la durée de préavis en cas de mise ...,2542-quelle-est-la-duree-de-preavis-en-cas-de-...,250,"collectif, un usage. si le salarié a entre 6 m..."
2,contributions,1008f18f84,"En cas de maladie, le salarié a-t-il droit à u...",1351-en-cas-de-maladie-le-salarie-a-t-il-droit...,128,le code du travail prévoit une protection diff...
3,contributions,101024d0ee,Quelle est la durée du congé de maternité ?,1486-quelle-est-la-duree-du-conge-de-maternite,40,l'employeur applique les conditions prévues pa...
4,contributions,1012262e3f,Est-il obligatoire d'avoir un contrat de trava...,1578-est-il-obligatoire-davoir-un-contrat-de-t...,790,le contrat de travail peut être verbal (donc n...
...,...,...,...,...,...,...
62,contributions,1a6a731104,Quelle est la durée maximale de la période d'e...,2579-quelle-est-la-duree-maximale-de-la-period...,914,le code du travail prévoit la durée maximale d...
63,contributions,1a6a731104,Quelle est la durée maximale de la période d'e...,2579-quelle-est-la-duree-maximale-de-la-period...,914,un accord d'entreprise en prévoit une. la duré...
64,contributions,1a6a731104,Quelle est la durée maximale de la période d'e...,2579-quelle-est-la-duree-maximale-de-la-period...,914,l'entreprise au cours des 3 mois précédant l'e...
65,contributions,1a6a731104,Quelle est la durée maximale de la période d'e...,2579-quelle-est-la-duree-maximale-de-la-period...,914,cette embauche est effectuée dans un emploi co...


In [7]:
fiches = pd.read_csv("/home/remi/Downloads/xp-documents-fiches (2).csv")

In [8]:
fiches["cleaned_text"] = fiches["text"].apply(lambda t : clean(re.sub("\s+", " ", str(t)), to_ascii=False))
fiches["tokens"] = fiches["cleaned_text"].apply(lambda t : len(str(t).split(" ")))


In [9]:
#fiches.sort_values(by=["tokens"], ascending=False)[20:40]
fiches = fiches[fiches["tokens"]<3000]

In [10]:
fiches["windows"] = fiches["cleaned_text"].apply(lambda t : make_windows(t, window_size, slide))
fiches.drop(columns=["cleaned_text", "text"], inplace=True)
fiches = fiches.explode("windows", ignore_index=True)
fiches.rename(columns={"windows": "text"}, inplace=True)

In [11]:
fiches

,source,cdtnId,title,slug,answers.generic.text,tokens,text
0,fiches_ministere_travail,100c1454c5-01,La prise en charge des frais de transport par ...,la-prise-en-charge-des-frais-de-transport-par-...,-,61,à savoir dans le contexte actuel de forte infl...
1,fiches_ministere_travail,100c1454c5-11,La prise en charge des frais de transport par ...,la-prise-en-charge-des-frais-de-transport-par-...,-,1406,l'employeur a l'obligation de prendre en charg...
2,fiches_ministere_travail,100c1454c5-11,La prise en charge des frais de transport par ...,la-prise-en-charge-des-frais-de-transport-par-...,-,1406,code du travail relatifs à la prise en charge ...
3,fiches_ministere_travail,100c1454c5-11,La prise en charge des frais de transport par ...,la-prise-en-charge-des-frais-de-transport-par-...,-,1406,"et militaires de l'état, des collectivités ter..."
4,fiches_ministere_travail,100c1454c5-11,La prise en charge des frais de transport par ...,la-prise-en-charge-des-frais-de-transport-par-...,-,1406,"à leur lieu de travail ; acheter, pour payer c..."
...,...,...,...,...,...,...,...
4723,fiches_service_public,fe23276022,Comment saisir le médiateur de l'apprentissage ?,comment-saisir-le-mediateur-de-lapprentissage,-,628,l'amiable une solution à leur problème. attent...
4724,fiches_service_public,fe23276022,Comment saisir le médiateur de l'apprentissage ?,comment-saisir-le-mediateur-de-lapprentissage,-,628,solution à leur litige en cours de contrat. ru...
4725,fiches_service_public,fe54f10a87,Droit du travail dans le secteur privé : qu'es...,droit-du-travail-dans-le-secteur-prive-quest-c...,-,135,la force majeure est un événement qui remplit ...
4726,fiches_service_public,fee7f610fa,Embauche d'un salarié par un groupement d'empl...,embauche-dun-salarie-par-un-groupement-demploy...,-,260,le salarié d'un groupement d'employeurs peut ê...


In [12]:
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-base')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-base')

In [13]:
# embbed each individually
def run_embeddings(elements):
    p_emb = []
    i = 0
    for p in elements :
        i+=1
        print(i)
        #print(sys.getsizeof(p_emb))
        batch_dict = tokenizer(p, max_length=512, padding=True, truncation=True, return_tensors='pt')
    
        outputs = model(**batch_dict)
        embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
        normalized_embeddings = F.normalize(embeddings, p=2, dim=1)
        print(batch_dict["input_ids"].size())
        p_emb.append(normalized_embeddings.detach())
    return torch.squeeze(torch.stack(p_emb))

In [153]:
fiches["query"] = fiches["text"].apply(lambda c : "query: " + c)
fiches_tensor = run_embeddings(fiches["query"])

1
torch.Size([1, 89])
2
torch.Size([1, 449])
3
torch.Size([1, 460])
4
torch.Size([1, 475])
5
torch.Size([1, 484])
6
torch.Size([1, 474])
7
torch.Size([1, 449])
8
torch.Size([1, 445])
9
torch.Size([1, 445])
10
torch.Size([1, 439])
11
torch.Size([1, 439])
12
torch.Size([1, 448])
13
torch.Size([1, 459])
14
torch.Size([1, 480])
15
torch.Size([1, 243])
16
torch.Size([1, 340])
17
torch.Size([1, 485])
18
torch.Size([1, 428])
19
torch.Size([1, 43])
20
torch.Size([1, 512])
21
torch.Size([1, 512])
22
torch.Size([1, 503])
23
torch.Size([1, 512])
24
torch.Size([1, 512])
25
torch.Size([1, 512])
26
torch.Size([1, 489])
27
torch.Size([1, 489])
28
torch.Size([1, 512])
29
torch.Size([1, 450])
30
torch.Size([1, 462])
31
torch.Size([1, 346])
32
torch.Size([1, 175])
33
torch.Size([1, 485])
34
torch.Size([1, 488])
35
torch.Size([1, 489])
36
torch.Size([1, 483])
37
torch.Size([1, 479])
38
torch.Size([1, 136])
39
torch.Size([1, 508])
40
torch.Size([1, 512])
41
torch.Size([1, 504])
42
torch.Size([1, 512])
43


In [14]:
#torch.save(fiches_tensor, '/home/remi/Desktop/fiches_tensor3.pt')


In [143]:
contribs["query"] = contribs["text"].apply(lambda c : "query: " + c)
contribs_tensor = run_embeddings(contribs["query"])

1
torch.Size([1, 351])
2
torch.Size([1, 254])
3
torch.Size([1, 169])
4
torch.Size([1, 61])
5
torch.Size([1, 480])
6
torch.Size([1, 503])
7
torch.Size([1, 495])
8
torch.Size([1, 501])
9
torch.Size([1, 204])
10
torch.Size([1, 141])
11
torch.Size([1, 247])
12
torch.Size([1, 348])
13
torch.Size([1, 251])
14
torch.Size([1, 48])
15
torch.Size([1, 61])
16
torch.Size([1, 96])
17
torch.Size([1, 195])
18
torch.Size([1, 6])
19
torch.Size([1, 442])
20
torch.Size([1, 453])
21
torch.Size([1, 461])
22
torch.Size([1, 470])
23
torch.Size([1, 132])
24
torch.Size([1, 390])
25
torch.Size([1, 292])
26
torch.Size([1, 341])
27
torch.Size([1, 340])
28
torch.Size([1, 146])
29
torch.Size([1, 61])
30
torch.Size([1, 479])
31
torch.Size([1, 409])
32
torch.Size([1, 455])
33
torch.Size([1, 451])
34
torch.Size([1, 442])
35
torch.Size([1, 462])
36
torch.Size([1, 455])
37
torch.Size([1, 446])
38
torch.Size([1, 390])
39
torch.Size([1, 292])
40
torch.Size([1, 416])
41
torch.Size([1, 313])
42
torch.Size([1, 446])
43
torch

In [15]:
#torch.save(contribs_tensor, '/home/remi/Desktop/contribs_tensor3.pt')

In [14]:
contribs_tensor = torch.load('/home/remi/Desktop/contribs_tensor3.pt')
fiches_tensor = torch.load('/home/remi/Desktop/fiches_tensor3.pt')
    

In [16]:
contribs_tensor.size()

torch.Size([67, 768])

In [15]:
tensor_docs = torch.vstack([fiches_tensor, contribs_tensor])
tensor_docs.size()

torch.Size([4795, 768])

In [19]:
passages_set = pd.concat([fiches, contribs]).reset_index()
passages_set.shape


(4795, 8)

In [19]:
questions_raw = read_excel("/home/remi/Downloads/questions-fulltext.xlsx", header=None).dropna()
questions = questions_raw[0].apply(lambda q : clean(str(q), to_ascii=False)).to_list()

In [158]:
#question_elements = [(lambda q : "query:" + q)(q) for q in questions]
#tensor_questions = run_embeddings(question_elements)

In [45]:
def search(query):
    # embbed query
    batch_dict = tokenizer('query: '+ query, max_length=512, padding=True, truncation=True, return_tensors='pt')

    outputs = model(**batch_dict)
    embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask']).detach()
    question_normalized_embeddings = F.normalize(embeddings, p=2, dim=1)

    # score against all passages and retrieve top k
    
    retrieval_scores = (question_normalized_embeddings @ tensor_docs.T)[0] * 100
    k = 20
    
    j = torch.argmax(retrieval_scores).item()
    matches = torch.topk(retrieval_scores, k).values.tolist()
    matching_indices = torch.topk(retrieval_scores, k).indices.tolist()
    #print(matches)
    #print(matching_indices)

    # reranker
    batch = ['query: '+ query]

    for m in matching_indices :    
        batch.append('passage: ' + passages_set.loc[m]["text"])
    
    #print(batch)
        
    batch_dict = tokenizer(batch, max_length=512, padding=True, truncation=True, return_tensors='pt')

    outputs = model(**batch_dict)
    embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask']).detach()
    normalized_embeddings = F.normalize(embeddings, p=2, dim=1)

    reranker_scores = (normalized_embeddings[0] @ normalized_embeddings[1:].T) * 100
    #print(reranker_scores)
        
    #print(reranker_scores)
    rr_matches = torch.topk(reranker_scores, k).values.tolist()
    rr_matching_indices = torch.topk(reranker_scores, k).indices.tolist()
    #print(len(rr_matching_indices))
    #print(rr_matching_indices)

    final_indices = [matching_indices[m] for m in rr_matching_indices]
    #print(final_indices)
    
    print(query)

    res = passages_set.loc[final_indices][["index","source","cdtnId","title","text", "tokens", "slug"]]
    
    res = res.head(7)
    
    display(res[["source", "cdtnId", "title", "text"]].style.set_properties(**{
        'text-align': 'left',
        'white-space': 'pre-wrap',
    }))

    return res
    
    
    # for rrm in rr_matching_indices :
    #     # -1 since first is query itself
    #     m = matching_indices[rrm]
    #     print(m)
    #     print("///////////////////////////////////////////////////////")
    
    #     print(passages_set.loc[m][0])
    #     print(passages_set.loc[m][1])
    #     print()
        
    # print()
    

In [46]:
search("combien de jours de congés pour naissance d'un gamin ?")

[3411, 3648, 3649, 3150, 3154, 3409, 3407, 3152, 3412, 3151, 3650, 3153, 3156, 3410, 3157, 3976, 3089, 3155, 3442, 3158]
[3649, 3152, 3648, 3156, 3154, 3153, 3650, 3155, 3157, 3411, 3151, 3409, 3150, 3412, 3410, 3407, 3158, 3442, 3976, 3089]
combien de jours de congés pour naissance d'un gamin ?


,source,cdtnId,title,text
3649,fiches_service_public,76ef24f331,Peut-on prendre un congé de paternité et d'accueil de l'enfant à la suite d'un congé de naissance ?,"la durée du congé de naissance est de 3 jours ouvrables . votre congé commence, selon votre choix : soit le jour de la naissance de votre enfant soit le 1 er jour ouvrable suivant le jour de la naissance. pour justifier de votre droit au congé de naissance, vous devez remettre à votre employeur une copie de l'acte de naissance de votre enfant. organiser le congé de paternité et d'accueil de l'enfant vous devez ensuite prendre votre congé de paternité et d'accueil de l'enfant. la durée de votre congé de paternité et d'accueil de votre enfant est de 25 jours calendaires (ou de 32 jours en cas de naissance multiple). le congé de paternité et d'accueil de l'enfant comporte une période obligatoire de 4 jours calendaires qui doit être prise immédiatement à la suite de votre congé de naissance. exemple un salarié dont l'enfant naît un samedi bénéficie d'un congé de naissance de 3 jours. le congé de naissance est calculé en jours ouvrables . le congé débute donc le 1 er jour ouvrable suivant la naissance, il a donc lieu du lundi au mercredi. le salarié doit également prendre immédiatement ses 4 jours obligatoires de congé de paternité et d'accueil de l'enfant. ce congé de paternité et d'accueil de l'enfant est décompté en jours calendaires , soit du jeudi au dimanche. le salarié doit donc prendre un congé cumulé du lundi au dimanche. il vous reste alors 21 jours calendaires (ou 28 jours en cas de naissance multiple) de congé de paternité et d'accueil de l'enfant. vous devez prendre impérativement ces 21 jours dans les 6 mois qui suivent la naissance de l'enfant. la seconde période de votre congé de 21 jours peut être fractionnée. vous pouvez prendre ces 21 jours en une seule fois ou en 2 périodes au plus. chacune des périodes doit comporter une durée"
3152,fiches_service_public,3f4a1c5605,Congé de paternité et d'accueil de l'enfant d'un salarié du secteur privé,"de l'enfant ? naissance d'un enfant la durée de votre congé de paternité et d'accueil de votre enfant est de 25 jours calendaires . votre congé comporte 2 périodes distinctes. 1 période obligatoire de 4 jours calendaires prise immédiatement après le congé de naissance vous êtes dans l'obligation de prendre votre congé de naissance de 3 jours et la 1 ere période de 4 jours de votre congé de paternité et d'accueil de l'enfant. exemple votre enfant naît un dimanche. vous bénéficiez d'un congé de naissance de 3 jours. le congé de naissance est calculé en jours ouvrables . le congé débute donc le 1 er jour ouvrable suivant la naissance, il a donc lieu du lundi au mercredi. vous devez également prendre immédiatement les 4 jours obligatoires de congé de paternité et d'accueil de l'enfant. ce congé de paternité et d'accueil de l'enfant est décompté en jours calendaires , soit du jeudi au dimanche. vous devez donc prendre un congé cumulé du lundi au dimanche. 1 période de 21 jours calendaires la seconde période de votre congé de 21 jours peut être fractionnée. vous pouvez prendre ces 21 jours en une seule fois ou en 2 périodes au plus. chacune des périodes doit comporter une durée minimale de 5 jours . exemple votre enfant naît le lundi 11 septembre 2023, vous pouvez prendre vos congés de la manière suivante : congé de naissance de 3 jours ouvrables du lundi 11 au mercredi 13 septembre 2023 inclus période obligatoire de congé de paternité et d'accueil de l'enfant de 4 jours calendaires du jeudi 14 au dimanche 17 septembre 2023 inclus période fractionnée de congé de paternité et d'accueil de l'enfant de 5 jours calendaires du lundi 25 au vendredi 29 septembre 2023 inclus puis de 16 jours calendaires du lundi 9 au mardi 24 octobre 2023 inclus naissance de plusieurs"
3648,fiches_service_public,76ef24f331,Peut-on prendre un congé de paternité et d'accueil de l'enfant à la suite d'un congé de naissance ?,"oui, vous pouvez prendr

,index,source,cdtnId,title,text,tokens,slug
3649,3649,fiches_service_public,76ef24f331,Peut-on prendre un congé de paternité et d'acc...,la durée du congé de naissance est de 3 jours ...,552,peut-on-prendre-un-conge-de-paternite-et-daccu...
3152,3152,fiches_service_public,3f4a1c5605,Congé de paternité et d'accueil de l'enfant d'...,de l'enfant ? naissance d'un enfant la durée d...,2236,conge-de-paternite-et-daccueil-de-lenfant-dun-...
3648,3648,fiches_service_public,76ef24f331,Peut-on prendre un congé de paternité et d'acc...,"oui, vous pouvez prendre votre congé de patern...",552,peut-on-prendre-un-conge-de-paternite-et-daccu...
3156,3156,fiches_service_public,3f4a1c5605,Congé de paternité et d'accueil de l'enfant d'...,de 5 jours calendaires du lundi 25 au vendredi...,2236,conge-de-paternite-et-daccueil-de-lenfant-dun-...
3154,3154,fiches_service_public,3f4a1c5605,Congé de paternité et d'accueil de l'enfant d'...,et d'accueil de l'enfant est décompté en jours...,2236,conge-de-paternite-et-daccueil-de-lenfant-dun-...
3153,3153,fiches_service_public,3f4a1c5605,Congé de paternité et d'accueil de l'enfant d'...,et d'accueil de l'enfant. exemple votre enfant...,2236,conge-de-paternite-et-daccueil-de-lenfant-dun-...
3650,3650,fiches_service_public,76ef24f331,Peut-on prendre un congé de paternité et d'acc...,d'accueil de l'enfant vous devez ensuite prend...,552,peut-on-prendre-un-conge-de-paternite-et-daccu...


In [40]:
res = []
for i in range(len(questions)):
    question = questions[i]
    res.append([question, search(question)])

NameError: name 'questions' is not defined

In [83]:
%%capture cap --no-stderr

doc = []
for q, r in res :
    doc.append(f'## {q}\n')
    doc.append("<details>\n")
    doc.append("<summary>Résultats</summary>\n")
    for group, frame in r.groupby(['cdtnId', 'source', 'title', 'slug'], sort=False) :        
        doc.append(f'#### {group[2]}\n')
        doc.append(f'source : {group[1]}\n')
        doc.append(f'slug {group[3]}\n')
        doc.append(f'cdtnId : {group[0]}\n')
        for t in frame["text"].tolist() :
            doc.append(f'> {t}\n')
    doc.append("</details>\n")

    doc.append('\n')
    
print("\n".join(doc))


with open('/home/remi/Desktop/output-xp', 'w+') as f:
    f.write(cap.stdout)


In [24]:
#res = search(questions[0])

In [65]:
queries_raw = read_excel("/home/remi/Downloads/Export.recherches.site.du.01-01-23.au.31-08-23.xlsx")

In [69]:
queries = queries_raw["Nom"].head(150).apply(lambda q : clean(str(q), to_ascii=False)).to_list()

In [70]:
queries_raw

,Nom,Recherches
0,procédure de licenciement salarié à domicile,23751
1,contrat de travail PARTICULIER EMPLOYEUR,10884
2,je veux démissionner,10537
3,congés payés,5838
4,3239,5591
...,...,...
495,paiement solde de tout compte,33
496,R4141-4,33
497,1090,32
498,3127,32


In [72]:
res_queries = []
for i in range(len(queries)):
    query = queries[i]
    res_queries.append([query, search(query)])

procédure de licenciement salarié à domicile


,source,title,text
1374,fiches_ministere_travail,Les droits et obligations des salariés du particulier employeur : bénéficier d’un certain nombre de garanties en cas de rupture du contrat de travail,"et les instruit ; sur le site de cet organisme, on trouvera une foire aux questions présentant l'ensemble des dispositions applicables (bénéficiaires, conditions à remplir, montant, procédure, versement etc.). s'il s'agit d'un licenciement, celui-ci doit reposer sur une cause réelle et sérieuse qui peut être inhérente au salarié (comportement fautif, mauvaise exécution du travail, etc.) ou relever du particulier employeur (baisse de revenus, départ en maison de retraite, etc.).l'employeur doit suivre la procédure prévue pour un licenciement individuel pour motif personnel (convocation à un entretien préalable, entretien préalable, notification du licenciement) telle qu'elle est précisée par l'article 161.1.1.1 de la ccn et remettre au salarié les documents de fin de contrat mentionnés à l'article 69 de la ccn et, si les conditions sont réunies, lui verser l'indemnité de licenciement prévue à l'article 163-1 de la ccn. les dispositions spécifiques applicables en cas de licenciement sont fixées par les articles 162.4.1 (préavis) et 162.4.2 (heures d'absence autorisée) de la ccn.les dispositions relatives à la rupture du contrat de travail à durée déterminée sont prévues par les dispositions légales et réglementaires du droit commun.en cas de mise à la retraite par l'employeur, le salarié bénéficie, quelle que soit son ancienneté, d'une indemnité de mise à la retraite dont le montant est calculé de la même manière que l'indemnité de licenciement prévue à l'article 163-1 de la ccn. décès du particulier employeur le décès du particulier employeur entraîne de plein droit la rupture du contrat de travail. les dispositions applicables (notification du décès par un ayant-droit, sommes à verser au salarié, documents de fin de contrat, etc.) figurent à l'article 161.4.1 de la ccn des particuliers employeurs et de l'emploi à domicile. toutefois, lorsque le recrutement d'un employé de maison est décidé conjointement par le couple et qu'il y a, dans les faits, un co-emploi qui s'illustre notamment par des consignes transmises indifféremment"
3315,fiches_service_public,Démission du salarié à domicile employé par un particulier,"la démission est un mode de rupture du cdi à l'initiative du salarié qui manifeste clairement la volonté de rompre son contrat. comment le salarié informe-t-il son employeur ? le salarié doit-il respecter un préavis en cas de démission ? quel est le point de départ du préavis en cas de démission ? le salarié touche-t-il des indemnités de son employeur en cas de démission ? touche-t-il des indemnités de pôle emploi ? nous faisons un point sur la réglementation. la possibilité de démissionner dépend du type de contrat de travail du salarié. cdi qu'est-ce que la démission ? la démission est une rupture du contrat de travail qui permet au salarié de quitter son emploi. le salarié doit manifester clairement et de façon non équivoque la volonté de rompre son contrat de travail. il n'a pas l'obligation de préciser le motif pour lequel il démissionne. en cas d'ambiguïté sur la volonté claire et non équivoque , le conseil de prud'hommes peut requalifier la démission en licenciement injustifié (ou sans cause réelle et sérieuse) . à savoir dans certaines conditions , l'employeur peut considérer l'abandon de poste comme une démission. comment le salarié à domicile informe-t-il son employeur de sa démision ? le salarié doit informer l'employeur de sa démission par lettre recommandée avec ar ou par lettre remise en main propre contre décharge . un modèle de lettre est disponible : lettre de démission du salarié employé à domicile par un particulier direction de l'information légale et administrative (dila) - première ministre le salarié du particulier employeur doit-il respecter un préavis en cas de démission ? oui . en règle générale,

contrat de travail particulier employeur


,source,title,text
4044,fiches_service_public,Particulier employeur : contrat de travail du salarié à domicile,"par un avenant écrit. le contrat de travail et ses éventuels avenants sont établis en 2 exemplaires qui sont datés, signés et paraphés par l'employeur et le salarié. chacun conserve un exemplaire du contrat de travail et de ses éventuels avenants. éléments obligatoires du contrat de travail le contrat de travail doit préciser les éléments suivants : identité et adresse des parties numéro d'identification employeur. si le numéro n'est pas encore connu au moment de la conclusion du contrat de travail, il est communiqué dès que possible par le particulier employeur au salarié. numéro de sécurité sociale du salarié date d'embauche assurance du véhicule du salarié emploi occupé durée de la période d'essai durée du travail lieu de travail rémunération incluant le salaire horaire et la date de payement mensuel du salaire indemnités et prestations jours fériés travaillés repos hebdomadaire congés payés nature de l'emploi occupé par le salarié absences du particulier employeur heures de présence responsable de jour heures de présence de nuit garde partagée la garde partagée est un mode de garde qui consiste à garder des enfants de 2 familles au domicile de l'une et/ou de l'autre famille. les modalités sont définies aux contrats de travail. chaque particulier employeur doit établir individuellement un contrat de travail écrit avec le salarié. les particuliers employeurs doivent prévoir, dans chaque contrat de travail, une clause identique précisant le lien avec l'autre particulier employeur. à savoir le contrat de travail conclu avec un jeune travailleur âgé de 16 à 18 ans est signé par ce dernier s'il dispose de l'autorisation écrite de son représentant légal. cette condition ne s'applique pas aux mineurs émancipés . période d'essai durée la période d'essai n'est pas obligatoire. si une période d'essai est prévue, elle et doit être écrite dans le contrat de travail ou la lettre d'engagement. la période d'essai peut être renouvelée 1 fois"
4043,fiches_service_public,Particulier employeur : contrat de travail du salarié à domicile,"obligatoires du contrat de travail, dont la période d'essai, le contrat peut être conclu jusqu'au dernier jour de la période d'essai. il peut être rédigé sur tout support écrit (papier libre, mail). en cas de transmission au salarié, par courriel, du contrat de travail signé par l'employeur, le salarié adresse un mail d'acceptation actant de son accord clair et non équivoque. toute modification du contrat de travail doit être faite par un avenant écrit. le contrat de travail et ses éventuels avenants sont établis en 2 exemplaires qui sont datés, signés et paraphés par l'employeur et le salarié. chacun conserve un exemplaire du contrat de travail et de ses éventuels avenants. éléments obligatoires du contrat de travail le contrat de travail doit préciser les éléments suivants : identité et adresse des parties numéro d'identification employeur. si le numéro n'est pas encore connu au moment de la conclusion du contrat de travail, il est communiqué dès que possible par le particulier employeur au salarié. numéro de sécurité sociale du salarié date d'embauche assurance du véhicule du salarié emploi occupé durée de la période d'essai durée du travail lieu de travail rémunération incluant le salaire horaire et la date de payement mensuel du salaire indemnités et prestations jours fériés travaillés repos hebdomadaire congés payés nature de l'emploi occupé par le salarié absences du particulier employeur heures de présence responsable de jour heures de présence de nuit garde partagée la garde partagée est un mode de garde qui consiste à garder des enfants de 2 familles au domicile de l'une et/ou de l'autre famille. les modalités sont définies aux contrats de travail. chaque particulier employeur doit établir individuellement un contrat de travail écrit avec le salarié. les particuliers employeurs doivent prév

je veux démissionner


...

,source,title,text
4483,fiches_service_public,Un employeur peut-il modifier les horaires de temps de travail d'un salarié à temps partiel ?,"oui , votre employeur peut modifier vos horaires de temps de travail si vous travaillez à temps partiel. toutefois, cela est possible si votre contrat de travail le prévoit ou pas . votre contrat de travail prévoit la possibilité de modification de vos horaires de travail si votre employeur souhaite modifier vos de horaires de travail, il doit vous prévenir impérativement en respectant un délai dit délai de prévenance . ce délai de prévenance peut être fixé par convention collective ou par un accord d'entreprise . il est alors d'au moins 3 jours ouvrés . en l'absence de convention collective ou d'accord d'entreprise, le délai de prévenance est de 7 jours ouvrés minimum. vous acceptez la demande de modification des horaires par votre employeur. toutefois, si vous refusez la modification, vous devez indiquer à votre employeur que ce changement d'horaires proposé est incompatible avec : soit vos obligations familiales (garde d'enfant pour un parent isolé, nécessité d'assister un membre de la famille gravement malade ou dépendant) soit la poursuite de vos études (enseignement scolaire ou supérieur) soit l'accomplissement d'une activité fixée par un autre employeur soit une activité professionnelle non salariée le contrat ne prévoit pas la modification des horaires si votre employeur souhaite modifier vos de horaires de travail, il doit vous prévenir impérativement en respectant un délai dit délai de prévenance . ce délai de prévenance peut être fixé par convention collective ou par un accord d'entreprise . il est alors d'au moins 3 jours ouvrés . en l'absence de convention collective ou d'accord d'entreprise, le délai de prévenance est de 7 jours ouvrés minimum. vous pouvez refuser la demande de votre employeur. votre refus ne peut pas être considéré comme une faute. il ne peut pas constituer un motif de licenciement."
4454,fiches_service_public,Modification du contrat de travail d'un salarié,"(cph) . durée du temps de travail aménagement des horaires de travail l'employeur peut fixer une nouvelle répartition ou un réaménagement des horaires de travail. exemple en cas de forte activité durant une période connue à l'avance, le salarié peut être amené à travailler plus de 35 heures durant certaines semaines, puis moins de 35 heures les semaines suivantes. la réduction du temps de travail avec maintien du salaire antérieur résultant d'un accord d'entreprise s'impose à tous les salariés. le salarié ne peut pas s'opposer à ce changement. toutefois, le changement ne doit pas être lié à un motif discriminatoire . le salarié peut saisir le conseil de prud'hommes (cph) . en cas de refus du salarié, l'employeur doit choisir entre les 2 options suivantes : soit renoncer à modifier le contrat soit engager une procédure de licenciement (pour cause réelle et sérieuse , voire pour faute grave ) à noter si les horaires de travail sont renseignés dans le contrat de travail, ils constituent un élément essentiel du contrat. l'accord du salarié est nécessaire pour les modifier. modification des horaires de travail lorsque la durée du temps de travail est établie dans le contrat de travail, l'employeur ne peut pas la modifier sans l'accord du salarié. c'est le cas pour les changements suivants : passage d'un horaire fixe à un horaire variable passage d'un horaire continu à un horaire discontinu passage d'un horaire de jour à un horaire de nuit ou inversement la réduction de la durée du travail décidée par l'employeur, lorsqu'elle s'accompagne d'une réduction de salaire, entraîne une modification du contrat de travail. le salarié est libre de refuser cette modification. l'employeur informe le salarié et lui laisse un délai de réflexion d' 1 mois à partir de la réception de la lettre recommandée avec accusé de réception (lrar). l'absence de réponse du salarié à la fin"
4168,fiches_service_public,Temps de travail du

code du travail


,source,title,text
4772,contributions,Travail du dimanche : quelle contrepartie ?,"le code du travail prévoit les conditions de travail du dimanche. il peut préciser, dans certains cas, les conditions de rémunération. l'employeur applique les conditions prévues par la convention ou l'accord collectif ou un usage, si elles sont plus favorables que le code du travail pour le salarié. le contrat de travail peut toujours prévoir des mesures plus favorables, qui s'appliqueront."
825,fiches_ministere_travail,Troubles musculo-squelettiques : que disent les textes sur les obligations de l’employeur vis-à-vis de la santé des salariés ?,"la législation française encadre la prévention et la protection des salariés au travail en matière de santé et de sécurité. plusieurs textes de référence sont importants à connaître pour un employeur.le code du travail chaque employeur doit prendre les mesures nécessaires pour assurer la sécurité et protéger la santé physique et mentale des salariés comme l'expliquent les articles l 4121-1 à -5 du code du travail. ces dispositions comprennent : des actions de prévention des risques professionnels des actions d'information et de formation la mise en place d'une organisation et des moyens adaptés une obligation de résultats pour l'employeur l'employeur doit veiller à l'adaptation de ces mesures pour obtenir un changement des circonstances et une amélioration des situations existantes. en france, la jurisprudence considère qu'il s'agit d'une obligation de sécurité de résultats et pas uniquement de moyens. en d'autres termes, la mise en conformité ne suffit pas. il est impératif de parvenir à l'effet escompté."
4730,contributions,"En cas de maladie, le salarié a-t-il droit à une garantie d’emploi ?","le code du travail prévoit une protection différente du salarié en cas de maladie ou accident non professionnel et en cas de maladie professionnelle ou accident du travail. le code du travail prévoit que l'employeur ne peut pas licencier le salarié en raison de son état de santé. s'il invoque la maladie du salarié, le licenciement est discriminatoire et donc nul. toutefois, l'employeur peut licencier le salarié dans certains cas. le code du travail prévoit une protection du salarié en cas de maladie professionnelle ou accident du travail. l'employeur applique les conditions prévues par la convention ou l'accord collectif ou un usage, si elles sont plus favorables que le code du travail pour le salarié. le contrat de travail peut toujours prévoir des mesures plus favorables, qui s'appliqueront."
331,fiches_ministere_travail,Les missions et les prérogatives de l’inspection du travail : quels sont les droits et obligations des agents de l’inspection du travail ?,"emprisonnement d'un an et d'une amende de 37 500 euros. en outre, les dispositions des articles l. 433-3, l. 433-5 et l. 433-6 du code pénal qui prévoient et répriment respectivement les actes de violences, d'outrages et de résistance contre les officiers de police judiciaire sont applicables à ceux qui se rendent coupables de faits de même nature à l'égard des inspecteurs du travail et des contrôleurs du travail.ils sont libres d'organiser et de conduire des contrôles à leur initiative et décident des suites à leur apporter.un code de déontologie du service public de l'inspection du travail, établi par le décret n° 2017-541 du 12 avril 2017, fixe les règles que doivent respecter ses agents ainsi que leurs droits dans le respect des prérogatives et garanties qui leurs sont accordées pour l'exercice de leurs missions définies notamment par les conventions n°81 et n° 129 de l'organisation internationale du travail (oit) « sur l'inspection du travail et les dispositions du code du travail relatives à l'inspection du travail ». les dispositions applicables figurent aux articles r. 8124-1 à r. 8124-33 du code du travail.les agents de l'inspection du travail sont tenus à divers devoirs tels que l'impartialité, et le secret professionnel.pour aller plus loin : d

absences


,source,title,text
2647,fiches_ministere_travail,Les congés payés : quelle est l’incidence de l’absence pour maladie sur le droit aux congés payés ?,"les absences pour maladie (non professionnelle) n'ouvrent pas droit à des congés payés, sauf dispositions conventionnelles contraires. en revanche, l'employeur ne saurait déduire du congé annuel les jours d'absence pour maladie."
3203,fiches_service_public,Un salarié peut-il s'absenter pour participer à une campagne électorale ?,"à une campagne électorale à condition qu'il soit candidat à un mandat parlementaire ou local. aucune condition d'ancienneté du salarié n'est exigée. ce droit est accordée au salarié candidat aux élections suivantes : municipales départementales régionales européennes assemblée de corse assemblée nationale sénat conseil de la métropole de lyon durée la durée d'absence varie en fonction de l'élection à laquelle le salarié est candidat, dans les conditions suivantes : nombre de jours ouvrables d'absence autorisés en fonction du type d'élections type d'élections nombre de jours ouvrables d'absence autorisés élections municipales 10 élections départementales ou régionales 10 élections européennes 10 élections au conseil de la métropole de lyon 10 élections à l'assemblée de corse 10 élections à l'assemblée nationale 20 élections au sénat 20 démarche le salarié doit avertir son employeur 24 heures au moins avant le début de chaque absence (par écrit ou par oral). l'employeur ne peut pas refuser la demande d'absence du salarié. chaque absence doit être au moins équivalente à une demi-journée entière. statut du salarié si le salarié le demande, la durée de ses absences peut être décomptée des congés payés, dans la limite des droits acquis à la date du 1 er tour de scrutin. lorsqu'elles ne sont pas décomptées des congés payés, les absences ne sont pas rémunérées. dans ce cas, elles peuvent donner lieu à récupération, en accord avec l'employeur. les jours d'absence, payés ou non , pour participation à une campagne électorale sont assimilés à une période de travail effectif . les jours d'absence sont pris en compte pour le calcul de l'ancienneté et des droits à congés payés."
3195,fiches_service_public,Élu local salarié : absence et crédits d'heures,"compte pour le calcul de l'ancienneté et des droits à congés payés. le salarié absent durant son mandat d'élu ne peut pas être sanctionné (licenciement, déclassement professionnel ou sanction disciplinaire). absence pour participer à une campagne éléctorale le salarié candidat à des élections peut bénéficier également d'une autorisation d'absence pour participer à la campagne électorale. conditions le salarié doit être candidat à un mandat local. aucune condition d'ancienneté du salarié n'est exigée. durée le nombre de jours ouvrables d'absence autorisés est fixé à 10 . démarches le salarié avertit son employeur 24 heures au moins avant le début de chaque absence (par écrit ou par oral). l'employeur ne peut pas refuser la demande d'absence du salarié, sauf si le délai de 24 heures n'est pas respecté. situation du salarié si le salarié le demande, la durée de ses absences peut être décomptée des congés payés, dans la limite des droits acquis à ce titre à la date du premier tour de scrutin. lorsqu'elles ne sont pas décomptées des congés payés, les absences ne sont pas rémunérées. dans ce cas, elles peuvent donner lieu à récupération, en accord avec l'employeur. les jours d'absence, payés ou non , pour participation à une campagne électorale sont assimilés à une période de travail effectif . ils sont pris en compte pour le calcul de l'ancienneté et des droits à congés payés. crédit d'heures bénéficiaires le salarié élu municipal bénéficie d'un crédit d'heures, qui peut être utilisé pour : la gestion administrative de la commune ou de l'organisme auprès duquel il la représente la préparation des réunions des instances où il siège durée la durée du crédit d'heures autorisé varie selon les fonctions exercées et la taille de l

In [74]:
%%capture cap --no-stderr

doc = []
for q, r in res_queries :
    doc.append(f'## {q}\n')
    doc.append("<details>\n")
    doc.append("<summary>Résultats</summary>\n")
    for group, frame in r.groupby(['cdtnId', 'source', 'title'], sort=False) :        
        doc.append(f'#### {group[2]}\n')
        doc.append(f'source : {group[1]}\n')
        doc.append(f'slug : {group[0]}\n')
        doc.append(f'cdtnId : {group[0]}\n')
        for t in frame["text"].tolist() :
            doc.append(f'> {t}\n')
    doc.append("</details>\n")

    doc.append('\n')
    
print("\n".join(doc))


with open('/home/remi/Desktop/output-xp-queries', 'w+') as f:
    f.write(cap.stdout)


In [76]:
res_queries[0]

['procédure de licenciement salarié à domicile',
       index                    source           cdtnId  \
 1374   1374  fiches_ministere_travail  7fc9a530e6-7281   
 3315   3315     fiches_service_public       4e0dedaa2e   
 3316   3316     fiches_service_public       4e0dedaa2e   
 2839   2839     fiches_service_public       21fd152b4b   
 4623   4623     fiches_service_public       f4a38d38cd   
 2841   2841     fiches_service_public       21fd152b4b   
 1372   1372  fiches_ministere_travail  7fc9a530e6-7281   
 
                                                   title  \
 1374  Les droits et obligations des salariés du part...   
 3315  Démission du salarié à domicile employé par un...   
 3316  Démission du salarié à domicile employé par un...   
 2839  Licenciement du salarié à domicile employé par...   
 4623  Peut-on contester un licenciement économique a...   
 2841  Licenciement du salarié à domicile employé par...   
 1372  Les droits et obligations des salariés du part... 

In [20]:
search("combien de jours de congés pour naissance d'un gamin ?")

combien de jours de congés pour naissance d'un gamin ?


,source,title,text
3649,fiches_service_public,Peut-on prendre un congé de paternité et d'accueil de l'enfant à la suite d'un congé de naissance ?,"la durée du congé de naissance est de 3 jours ouvrables . votre congé commence, selon votre choix : soit le jour de la naissance de votre enfant soit le 1 er jour ouvrable suivant le jour de la naissance. pour justifier de votre droit au congé de naissance, vous devez remettre à votre employeur une copie de l'acte de naissance de votre enfant. organiser le congé de paternité et d'accueil de l'enfant vous devez ensuite prendre votre congé de paternité et d'accueil de l'enfant. la durée de votre congé de paternité et d'accueil de votre enfant est de 25 jours calendaires (ou de 32 jours en cas de naissance multiple). le congé de paternité et d'accueil de l'enfant comporte une période obligatoire de 4 jours calendaires qui doit être prise immédiatement à la suite de votre congé de naissance. exemple un salarié dont l'enfant naît un samedi bénéficie d'un congé de naissance de 3 jours. le congé de naissance est calculé en jours ouvrables . le congé débute donc le 1 er jour ouvrable suivant la naissance, il a donc lieu du lundi au mercredi. le salarié doit également prendre immédiatement ses 4 jours obligatoires de congé de paternité et d'accueil de l'enfant. ce congé de paternité et d'accueil de l'enfant est décompté en jours calendaires , soit du jeudi au dimanche. le salarié doit donc prendre un congé cumulé du lundi au dimanche. il vous reste alors 21 jours calendaires (ou 28 jours en cas de naissance multiple) de congé de paternité et d'accueil de l'enfant. vous devez prendre impérativement ces 21 jours dans les 6 mois qui suivent la naissance de l'enfant. la seconde période de votre congé de 21 jours peut être fractionnée. vous pouvez prendre ces 21 jours en une seule fois ou en 2 périodes au plus. chacune des périodes doit comporter une durée"
3152,fiches_service_public,Congé de paternité et d'accueil de l'enfant d'un salarié du secteur privé,"de l'enfant ? naissance d'un enfant la durée de votre congé de paternité et d'accueil de votre enfant est de 25 jours calendaires . votre congé comporte 2 périodes distinctes. 1 période obligatoire de 4 jours calendaires prise immédiatement après le congé de naissance vous êtes dans l'obligation de prendre votre congé de naissance de 3 jours et la 1 ere période de 4 jours de votre congé de paternité et d'accueil de l'enfant. exemple votre enfant naît un dimanche. vous bénéficiez d'un congé de naissance de 3 jours. le congé de naissance est calculé en jours ouvrables . le congé débute donc le 1 er jour ouvrable suivant la naissance, il a donc lieu du lundi au mercredi. vous devez également prendre immédiatement les 4 jours obligatoires de congé de paternité et d'accueil de l'enfant. ce congé de paternité et d'accueil de l'enfant est décompté en jours calendaires , soit du jeudi au dimanche. vous devez donc prendre un congé cumulé du lundi au dimanche. 1 période de 21 jours calendaires la seconde période de votre congé de 21 jours peut être fractionnée. vous pouvez prendre ces 21 jours en une seule fois ou en 2 périodes au plus. chacune des périodes doit comporter une durée minimale de 5 jours . exemple votre enfant naît le lundi 11 septembre 2023, vous pouvez prendre vos congés de la manière suivante : congé de naissance de 3 jours ouvrables du lundi 11 au mercredi 13 septembre 2023 inclus période obligatoire de congé de paternité et d'accueil de l'enfant de 4 jours calendaires du jeudi 14 au dimanche 17 septembre 2023 inclus période fractionnée de congé de paternité et d'accueil de l'enfant de 5 jours calendaires du lundi 25 au vendredi 29 septembre 2023 inclus puis de 16 jours calendaires du lundi 9 au mardi 24 octobre 2023 inclus naissance de plusieurs"
3648,fiches_service_public,Peut-on prendre un congé de paternité et d'accueil de l'enfant à la suite d'un congé de naissance ?,"oui, vous pouvez prendre votre congé de paternité et d'accueil 

,index,source,cdtnId,title,text,tokens,slug
3649,3649,fiches_service_public,76ef24f331,Peut-on prendre un congé de paternité et d'acc...,la durée du congé de naissance est de 3 jours ...,552,peut-on-prendre-un-conge-de-paternite-et-daccu...
3152,3152,fiches_service_public,3f4a1c5605,Congé de paternité et d'accueil de l'enfant d'...,de l'enfant ? naissance d'un enfant la durée d...,2236,conge-de-paternite-et-daccueil-de-lenfant-dun-...
3648,3648,fiches_service_public,76ef24f331,Peut-on prendre un congé de paternité et d'acc...,"oui, vous pouvez prendre votre congé de patern...",552,peut-on-prendre-un-conge-de-paternite-et-daccu...
3156,3156,fiches_service_public,3f4a1c5605,Congé de paternité et d'accueil de l'enfant d'...,de 5 jours calendaires du lundi 25 au vendredi...,2236,conge-de-paternite-et-daccueil-de-lenfant-dun-...
3154,3154,fiches_service_public,3f4a1c5605,Congé de paternité et d'accueil de l'enfant d'...,et d'accueil de l'enfant est décompté en jours...,2236,conge-de-paternite-et-daccueil-de-lenfant-dun-...
3153,3153,fiches_service_public,3f4a1c5605,Congé de paternité et d'accueil de l'enfant d'...,et d'accueil de l'enfant. exemple votre enfant...,2236,conge-de-paternite-et-daccueil-de-lenfant-dun-...
3650,3650,fiches_service_public,76ef24f331,Peut-on prendre un congé de paternité et d'acc...,d'accueil de l'enfant vous devez ensuite prend...,552,peut-on-prendre-un-conge-de-paternite-et-daccu...


In [21]:
reqres = pd.read_json("/home/remi/Downloads/Requêtes et résultats attendus.json")

Un contrat écrit est-il obligatoire pour un CDD ?


,source,cdtnId,title,text
4275,fiches_service_public,c6c206dc8b,Le contrat de travail est-il obligatoirement écrit ?,"le salarié est présumé être en cdi à temps plein. la requalification du contrat (c'est-à-dire sa transformation en cdi à temps plein) peut être actée par le conseil de prud'hommes (cph) à la demande du salarié. si l'employeur n'a pas fait d'écrit alors qu'il aurait dû le faire, il risque une amende d'un montant de 1 500 € pour un contrat à temps partiel ou intermittent. l'employeur risque également une amende de 3 750 € ( 7 500 € en cas de récidive) s'il recourt au cdd ou au travail temporaire sans contrat écrit. en règle général, un écrit n'est pas exigé pour un cdi à temps plein, mais une convention collective peut prévoir un écrit. en l'absence d'écrit obligatoire, l'employeur doit toutefois remettre au salarié une copie de la déclaration préalable à l'embauche (dpae) . en pratique et notamment pour prévenir tout conflit, la signature d'un contrat est toujours préférable. à savoir le contrat de travail constaté par écrit doit être rédigé en français. le salarié étranger peut demander la traduction de son contrat dans sa langue d'origine."
1450,fiches_ministere_travail,86de7caf81-7681,Le contrat à durée déterminée à objet défini : quels sont la forme et le contenu du contrat ?,"le cdd à objet défini est établi par écrit et comporte les clauses obligatoires pour les contrats de travail à durée déterminée. il comporte également les mentions suivantes : la mention « contrat à durée déterminée à objet défini » ; l'intitulé et les références de l'accord collectif qui institue ce contrat ; une clause descriptive du projet et la mention de sa durée prévisible ; la définition des tâches pour lesquelles le contrat est conclu ; l'événement ou le résultat objectif déterminant la fin de la relation contractuelle ; le délai de prévenance de l'arrivée au terme du contrat et, le cas échéant, de la proposition de poursuite de la relation de travail en contrat à durée indéterminée ; une clause mentionnant la possibilité de rupture à la date anniversaire de la conclusion du contrat par l'une ou l'autre partie pour un motif réel et sérieux et le droit pour le salarié, lorsque cette rupture est à l'initiative de l'employeur, à une indemnité égale à 10 % de la rémunération totale brute du salarié."
4274,fiches_service_public,c6c206dc8b,Le contrat de travail est-il obligatoirement écrit ?,"les contrats de travail suivants doivent obligatoirement être conclus par écrit : contrat à durée déterminée (cdd) contrat à durée indéterminée (cdi) conclu à temps partiel contrat de travail temporaire contrat de travail intermittent contrat d'apprentissage contrat de professionnalisation contrat unique d'insertion (cui) contrats conclus avec un groupement d'employeurs contrat de travail en portage salarial à durée déterminée ou indéterminée si le contrat n'est pas écrit alors qu'il devrait l'être, le salarié est présumé être en cdi à temps plein. la requalification du contrat (c'est-à-dire sa transformation en cdi à temps plein) peut être actée par le conseil de prud'hommes (cph) à la demande du salarié. si l'employeur n'a pas fait d'écrit alors qu'il aurait dû le faire, il risque une amende d'un montant de 1 500 € pour un contrat à temps partiel ou intermittent. l'employeur risque également une amende de 3 750 € ( 7 500 € en cas de récidive) s'il recourt au cdd ou au travail temporaire sans contrat écrit. en règle général, un écrit n'est pas exigé pour un cdi à temps plein, mais une convention collective peut prévoir un écrit. en l'absence d'écrit obligatoire, l'employeur doit toutefois remettre au salarié une copie de la déclaration préalable à l'embauche (dpae) . en pratique et notamment pour prévenir tout conflit, la signature d'un contrat est toujours préférable. à savoir le contrat de travail constaté par écrit doit être rédigé en français. le salarié étranger peut demander la traduction de son contrat dans sa langue d'origine."
1162,fic

[{'slug': 'fiche-service-public/le-contrat-de-travail-est-il-obligatoirement-ecrit',
  'cdtnId': 'c6c206dc8b'},
 {'slug': 'fiche-ministere-travail/le-contrat-a-duree-determinee-cdd',
  'cdtnId': '4e7740a027'},
 {'slug': 'information/informations-principales-sur-la-relation-de-travail-a-partir-du-1er-novembre-2023-quels-elements-doit-fournir-lemployeur-au-salarie-lors-de-son-embauche',
  'cdtnId': 'd77c8c5155'}]

[{'slug': 'contribution/si-le-salarie-est-malade-pendant-ses-conges-quelles-en-sont-les-consequences',
  'cdtnId': '8dffdbc6b9'},
 {'slug': 'fiche-service-public/un-salarie-peut-il-reporter-des-jours-de-conges-payes-pour-cause-de-maladie',
  'cdtnId': '6e888b3f6e'},
 {'slug': 'fiche-ministere-travail/les-conges-payes', 'cdtnId': 'fcf7934672'}]

In [33]:
res

,index,source,cdtnId,title,text,tokens,slug
2671,2671,fiches_service_public,10756d3a39,Peut-on prendre directement des congés après u...,"oui, vous pouvez prendre vos congés directemen...",81,peut-on-prendre-directement-des-conges-apres-u...
2647,2647,fiches_ministere_travail,fcf7934672-13911,Les congés payés : quelle est l’incidence de l...,les absences pour maladie (non professionnelle...,31,les-conges-payes#Quelle-est-l-incidence-de-l-a...
3364,3364,fiches_service_public,52562fc16f,Un salarié peut-il démissionner pendant ses co...,"oui, si vous êtes salarié dans le secteur priv...",75,un-salarie-peut-il-demissionner-pendant-ses-co...
3568,3568,fiches_service_public,6e888b3f6e,Arrêt maladie et congés payés annuels,que se passe-t-il si vous tombez malade pendan...,741,arret-maladie-et-conges-payes-annuels
3570,3570,fiches_service_public,6e888b3f6e,Arrêt maladie et congés payés annuels,congés si vous êtes malade (maladie non-profes...,741,arret-maladie-et-conges-payes-annuels
3571,3571,fiches_service_public,6e888b3f6e,Arrêt maladie et congés payés annuels,prise de congés en cours dans l'entreprise ou ...,741,arret-maladie-et-conges-payes-annuels
3569,3569,fiches_service_public,6e888b3f6e,Arrêt maladie et congés payés annuels,de congés restants si la convention collective...,741,arret-maladie-et-conges-payes-annuels
